<a href="https://colab.research.google.com/github/nneko1231/RoopColab/blob/main/DeepFake_V2.4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Drive Mount

In [ ]:
# will mount drive if MOUNT_DRIVE is True
import os
import subprocess
import shutil
from google.colab import drive, output

# Add a parameter to control mounting Google Drive
MOUNT_DRIVE = True  #@param {type:"boolean"}

# mount drive if MOUNT_DRIVE is True
if MOUNT_DRIVE:
    drive.mount('/content/drive')

# 2. Installing

In [ ]:
# =========================================
# INSTALL PYTHON 3.10 + PIP
# =========================================
!sudo apt-get update
!sudo apt-get install python3.10 python3.10-distutils -y
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10

print("Python & pip version:")
!python3.10 --version
!python3.10 -m pip --version


# =========================================
# SELECT EXECUTION ENVIRONMENT
# =========================================
Processing_Use = "GPU"  # @param ["GPU", "CPU"]


# =========================================
# CLONE ROOP
# =========================================
!git clone https://github.com/nneko1231/roop
%cd roop


# =========================================
# INSTALL ONNXRUNTIME
# =========================================
if Processing_Use == "GPU":
    print("You have selected GPU. Installing GPU-specific libraries...")
    !python3.10 -m pip install onnxruntime-gpu
else:
    print("You have selected CPU. Installing CPU-specific libraries...")
    !python3.10 -m pip install onnxruntime


# =========================================
# INSTALL REQUIREMENTS (PYTHON 3.10)
# =========================================
!python3.10 -m pip install -r requirements.txt


# =========================================
# PATCH JAX (SAFE FOR PY 3.10)
# =========================================
file_path = "/usr/local/lib/python3.10/dist-packages/jax/_src/dtypes.py"

with open(file_path, "r") as f:
    content = f.read()

content = content.replace("np.dtypes", "np.dtype")

with open(file_path, "w") as f:
    f.write(content)

print("JAX patch applied successfully")

In [ ]:
#@title Model Download
!wget https://huggingface.co/nneko1231/inswapper/resolve/main/inswapper_128.onnx -O inswapper_128.onnx
!mkdir models
!mv inswapper_128.onnx ./models

# 3. Settings

In [ ]:
SOURCE_PATH = "SOURCE.jpg" #@param {type:"string"}
TARGET_PATH = "TARGET.mp4" #@param {type:"string"}
OUTPUT_PATH = "OUTPUT.mp4" #@param {type:"string"}
#@markdown - Recommended to use face_swapper if only using CPU.
frame_processor = "face_swapper face_enhancer" # @param ["face_swapper face_enhancer","face_swapper","face_enhancer"]
#@markdown - Recommended GPU 8, CPU 2
execution_threads = 8 #@param {type:"slider", min:1, max:8, step:1}
keep_fps = True  #@param {type:"boolean"}
keep_frames = True  #@param {type:"boolean"}
skip_audio = False  #@param {type:"boolean"}
temp_frame_format = "png" #@param ["jpg" , "png"]
#@markdown - Recommended 80–95 to balance quality and efficiency.
temp_frame_quality = 80 # @param {"type":"slider","min":0,"max":100,"step":1}
output_video_encoder = "h264_nvenc" # @param ["libx264","libx265","libvpx-vp9","h264_nvenc","hevc_nvenc"]
#@markdown - For professional or publication results, a value of 80+ is recommended.
#@markdown - For a quick test or preview, a value of 40–60 can be used.
output_video_quality = 80 # @param {"type":"slider","min":0,"max":100,"step":1}
max_memory = 12 # @param {"type":"slider","min":0.1,"max":12,"step":0.1}


# 4. Run


In [ ]:
#@title Run Roop
import os

def build_backend_command() -> str:
    # Start building the command with basic parameters
    command = (
        'python3.10 /content/roop/run.py' +
        f' -s "{SOURCE_PATH}" ' +
        f' -t "{TARGET_PATH}" ' +
        f' -o "{OUTPUT_PATH}" ' +
        f' --execution-threads {execution_threads}'  # Default number of execution threads
    )

    # List of encoders that do not require CUDA
    non_cuda_encoders = ["libx264", "libx265", "libvpx-vp9"]

    # If the selected encoder is one of the non-CUDA encoders
    if output_video_encoder in non_cuda_encoders:
        # Disable CUDA and run with CPU
        command += ' --execution-provider cpu '
    else:
        # Use CUDA for encoding (for encoders like h264_nvenc, hevc_nvenc)
        command += ' --execution-provider cuda '

    # Always add the selected frame processors
    command += f' --frame-processor {frame_processor}'

    # Add options if they are set to True
    if keep_fps:
        command += ' --keep-fps '
    if keep_frames:
        command += ' --keep-frames '
    if skip_audio:
        command += ' --skip-audio '

    # Add the other parameters for frame format, video encoder, etc.
    command += (
        f' --temp-frame-format {temp_frame_format} ' +
        f' --temp-frame-quality {temp_frame_quality} ' +
        f' --output-video-encoder {output_video_encoder} ' +
        f' --output-video-quality {output_video_quality} ' +
        f' --max-memory {max_memory} '
    )
    return command

try:
    # Write the command to a shell script file
    with open("command.sh", "w") as f:
        f.write(build_backend_command())

    # Execute the command from the shell script
    !bash command.sh

except KeyboardInterrupt:
    print("The process was terminated by the user.")

finally:
    # Ensure the command.sh file is deleted after execution
    if os.path.exists("command.sh"):
        os.remove("command.sh")
        print("The command.sh file is deleted.")

# 5. Best Output

In [ ]:
import cv2
import os
from moviepy.editor import VideoFileClip, AudioFileClip
from tqdm import tqdm  # Import tqdm for progress bar

# Folder location for images and output file
input_folder = "INPUT"  # @param {"type":"string"}
output_video = "OUTPUT.mp4"  # @param {"type":"string"}
fps = 30  # @param {"type":"number"}

audio = True  # @param {"type":"boolean"}

# Audio file to be added
audio_file = "AUDIO"  # @param {"type":"string"}

# Output video file with audio
output_video_with_audio = "OUTPUT_with_audio.mp4"  # @param {"type":"string"}

# Valid image extensions
valid_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.gif')

# Get list of image files in the folder, sorted by name
images = [img for img in sorted(os.listdir(input_folder)) if img.lower().endswith(valid_extensions)]

# Check if the folder is empty or has no valid images
if not images:
    print("The folder is empty or does not contain valid image files!")
    exit()

# Get the dimensions of the first image (assuming all images have the same size)
img_path = os.path.join(input_folder, images[0])
img = cv2.imread(img_path)
height, width, layers = img.shape

# Create a VideoWriter object to write the video
file_extension = os.path.splitext(output_video)[1].lower()

if file_extension == '.mp4':
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
elif file_extension == '.avi':
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
else:
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Default codec

video_writer = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

# Add each image to the video with a progress bar
for i, image in tqdm(enumerate(images), total=len(images), desc="Creating Video"):
    img_path = os.path.join(input_folder, image)
    img = cv2.imread(img_path)
    video_writer.write(img)

# Finished, close VideoWriter
video_writer.release()

# Adding music to the video using moviepy if 'audio' is True
if audio:
    try:
        # Load the video that was created
        video_clip = VideoFileClip(output_video)

        # Check if the audio_file is an audio or video file
        if audio_file.lower().endswith(('.mp3', '.wav', '.ogg', '.flac')):
            # If the audio file is MP3, WAV, etc.
            audio_clip = AudioFileClip(audio_file)
        elif audio_file.lower().endswith(('.mp4', '.avi', '.mov')):
            # If the audio is from a video file (e.g., MP4), extract the audio from the video
            video_audio_clip = VideoFileClip(audio_file).audio
            audio_clip = video_audio_clip
        else:
            raise ValueError("Invalid audio file. Make sure it's .mp3, .wav, or a video file (.mp4, .avi).")

        # Adjust the audio duration to match the video duration
        audio_clip = audio_clip.subclip(0, video_clip.duration)

        # Force synchronization of audio and video
        video_clip = video_clip.set_audio(audio_clip)

        # Save the video with audio with a progress bar
        video_clip.write_videofile(output_video_with_audio, codec="libx264", audio_codec="aac", audio=True, threads=4)

        print(f"Video with audio successfully created: {output_video_with_audio}")

        # Delete the original video (output_video) after successfully creating the video with audio
        if os.path.exists(output_video):
            os.remove(output_video)
            print(f"Original video {output_video} has been deleted.")

    except Exception as e:
        print(f"An error occurred: {e}")
else:
    print("Audio is set to False, skipping the audio addition step.")
